In [1]:
!pip install openprompt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 146 kB 23.4 MB/s 
     |████████████████████████████████| 125 kB 74.2 MB/s 
     |████████████████████████████████| 5.8 MB 62.2 MB/s 
     |████████████████████████████████| 452 kB 77.0 MB/s 
     |████████████████████████████████| 1.2 MB 61.7 MB/s 
     |████████████████████████████████| 7.6 MB 61.3 MB/s 
     |████████████████████████████████| 182 kB 81.9 MB/s 
     |████████████████████████████████| 212 kB 76.0 MB/s 
     |████████████████████████████████| 132 kB 64.2 MB/s 
     |████████████████████████████████| 127 kB 77.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.metrics import f1_score, precision_score, recall_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv("/content/drive/MyDrive/uOttawa/S1/NLP/project2/train_top_10_pp.csv")
df.head()

,Unnamed: 0,Title,Tags,tag_count,description,code
0,0,how check uploaded file image without mime type,php,1,id like check uploaded file image file eg png ...,NaN
1,1,how i replace special characters url,c#,1,this probably simple i simply cannot find answ...,NaN
2,2,how fetch xml feed using aspnet,c# asp.net,2,ive decided convert windows phone app fetches ...,NaN
3,3,php framework url conventions,php,1,a lot frameworks use url conventions like grea...,/controller/action/{id}
4,4,play framework auto javascript css minifier,javascript,1,does anyone know good play plugin automaticall...,NaN


In [5]:
print(df.shape)

df["code"] = df["code"].fillna("")
df["description"] = df["description"].fillna("")
df = df[df['Title'].notna()]
print(df.shape)

(1898097, 6)
(1898077, 6)


In [6]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split())
tag_bow = vectorizer.fit_transform(df['Tags'])

In [7]:
tags = vectorizer.get_feature_names()
tags

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['android',
 'asp.net',
 'c#',
 'c++',
 'iphone',
 'java',
 'javascript',
 'jquery',
 'php',
 'python']

In [8]:
tags[3] = "c+"
tags

['android',
 'asp.net',
 'c#',
 'c+',
 'iphone',
 'java',
 'javascript',
 'jquery',
 'php',
 'python']

In [9]:
new_df = df.loc[df['Tags'].str.contains("c+")]
len(new_df)

722903

**Extract 2000 Sentences for each tag**

In [10]:
new_df = df.loc[df['Tags'].str.contains('android', case=False)]
print(len(new_df))
dataset=new_df[:2000]
#print(len(dataset))
for tag in tags:
  new_df = df.loc[df['Tags'].str.contains(tag)]
  new_df = new_df[:2000]
  print(len(new_df))
  dataset = dataset.append(new_df)
  dataset = dataset.drop_duplicates(keep='first')
  print(tag," ",len(dataset))


232412
2000
android   2000
2000
asp.net   3998
2000
c#   5717
2000
c+   6786
2000
iphone   8759
2000
java   9863
2000
javascript   10852
2000
jquery   12159
2000
php   13905
2000
python   15862


In [11]:
dataset.head()

,Unnamed: 0,Title,Tags,tag_count,description,code
13,13,loadable kernel module notification manager,android,1,ive successfully created installed loadable ke...,
45,45,eclipse indigo allow override non activity ove...,android,1,i using eclipse indigo android development the...,
52,52,android v webview loaddatawithbaseurl load page,android,1,when im loading page using system load page on...,.loadData
54,54,android changing locale within app affect firs...,java android,2,i use code bellow setcontentview main activity...,"Locale locale = new Locale(""ar"");"
58,58,android client socket unknowhostexception,java android,2,i working server application java multithreade...,


In [12]:
tags[3] = "c++"
tags

['android',
 'asp.net',
 'c#',
 'c++',
 'iphone',
 'java',
 'javascript',
 'jquery',
 'php',
 'python']

In [13]:
x_train, x_test, y_train, y_test = train_test_split(dataset, dataset['Tags'], test_size = 0.005, random_state = 42)

print("Number of data points in training data :", x_train.shape[0])
print("Number of data points in test data :", x_test.shape[0])

Number of data points in training data : 15782
Number of data points in test data : 80


In [14]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(), binary='true')
y_multilabel = vectorizer.fit_transform(y_test)

In [15]:
x_train.head()

,Unnamed: 0,Title,Tags,tag_count,description,code
12764,12764,doctrine error,php,1,as part application im working on im presentin...,$model-&gt;profile-&gt;populate($_POST['Accoun...
11805,11805,calling function custom function outside,javascript jquery,2,ive asked question find call function within c...,prepareSlide
10596,10596,nullpointeexception startactivity button,android,1,im trying launch actvivity via onclicklistener...,
24438,24438,how ensure user accepts terms conditions aspne...,asp.net,1,were modifying existing aspnet application we ...,
4780,4780,secure password hashing salt storing local coo...,c# asp.net,2,ive recently read interesting article securely...,ValidatePassword


# Prompt Learning

In [16]:
from openprompt.data_utils import InputExample

/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [17]:
classes = tags
dataset_p=[]
for i, row in x_test.iterrows():
  dataset_p.append(InputExample(guid = i, text_a = row["Title"]))


In [18]:
from openprompt.plms import load_plm
plm, tokenizer, model_config, WrapperClass = load_plm("roberta", "roberta-base")

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [19]:
from openprompt.prompts import ManualTemplate
promptTemplate = ManualTemplate(
    text = '{"placeholder":"text_a"} It was {"mask"}',
    tokenizer = tokenizer,
)

In [20]:
from openprompt.prompts import ManualVerbalizer
promptVerbalizer = ManualVerbalizer(
    classes = classes,
    label_words = classes ,
    tokenizer = tokenizer,
)

In [21]:
from openprompt import PromptForClassification
promptModel = PromptForClassification(
    template = promptTemplate,
    plm = plm,
    verbalizer = promptVerbalizer,
)

In [22]:
from openprompt import PromptDataLoader
data_loader = PromptDataLoader(
        dataset = dataset_p,
        tokenizer = tokenizer,
        template = promptTemplate,
        tokenizer_wrapper_class=WrapperClass,
    )

tokenizing: 80it [00:00, 1293.52it/s]


In [23]:
import numpy as np
def sig(x):
 return 1/(1 + np.exp(-x))

In [24]:
import torch
predict=[]
predict = torch.tensor(predict)
Total_labels=[]
i  = 0
promptModel.eval()
with torch.no_grad():
     for batch in data_loader:
         print("batch",i)
         i += 1
         logits = promptModel(batch)
         output = logits[0]
         sigmoid = sig(output)

         preds = torch.round(sigmoid, decimals=2) 
         
         #define a threshold
         indices = preds >= 0.2
         preds[indices] = 1
         preds[~indices] = 0
         
         index = [i for i in range(len(preds)) if (preds[i] == 1)]
         label = list(map(lambda x: classes[x],index))
         Total_labels.append(label)
        
         predict = torch.cat((predict, preds),-1)

         final_predict = predict.reshape(i,10)


batch 0
batch 1
batch 2
batch 3
batch 4
batch 5
batch 6
batch 7
batch 8
batch 9
batch 10
batch 11
batch 12
batch 13
batch 14
batch 15
batch 16
batch 17
batch 18
batch 19
batch 20
batch 21
batch 22
batch 23
batch 24
batch 25
batch 26
batch 27
batch 28
batch 29
batch 30
batch 31
batch 32
batch 33
batch 34
batch 35
batch 36
batch 37
batch 38
batch 39
batch 40
batch 41
batch 42
batch 43
batch 44
batch 45
batch 46
batch 47
batch 48
batch 49
batch 50
batch 51
batch 52
batch 53
batch 54
batch 55
batch 56
batch 57
batch 58
batch 59
batch 60
batch 61
batch 62
batch 63
batch 64
batch 65
batch 66
batch 67
batch 68
batch 69
batch 70
batch 71
batch 72
batch 73
batch 74
batch 75
batch 76
batch 77
batch 78
batch 79


In [25]:
print("prediction shape: ", final_predict.shape)

prediction shape:  torch.Size([80, 10])


In [26]:
final_predict[5]

tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 1.])

In [27]:
print("y_test shape: ", y_multilabel.shape)
y_test_T = y_multilabel.todense()
y_test_T = torch.tensor(y_test_T).type(torch.FloatTensor)
print("y_test tensor: ", y_test_T.shape)

y_test shape:  (80, 10)
y_test tensor:  torch.Size([80, 10])


In [28]:
print("Accuracy: ", metrics.accuracy_score(y_test_T, final_predict))
print("Macro f1 score: ", metrics.f1_score(y_test_T, final_predict, average = 'macro'))
print("Micro f1 scoore: ", metrics.f1_score(y_test_T, final_predict, average = 'micro'))
print("Hamming loss: ", metrics.hamming_loss(y_test_T, final_predict))

Accuracy:  0.1875
Macro f1 score:  0.14975414781297133
Micro f1 scoore:  0.22564102564102562
Hamming loss:  0.18875


**Results of 80 Samples**

In [50]:
i=0
with torch.no_grad():
     for batch in data_loader:
       
       print(tokenizer.decode(batch['input_ids'][0], skip_special_tokens=True), Total_labels[i])
       i += 1

jquery form plugin post dynamic fields It was ['python']
performance knock using runatserver html controls It was ['python']
emacs regexp syntax highlighting It was ['python']
android voice recognition arrays It was ['android', 'python']
unable find entry point named logonuser dll advapi dll impersonation exception It was ['java']
i see output errorlog die php It was ['javascript', 'python']
android buttons part larger png graphic It was ['javascript']
mvc procedure question It was ['asp.net']
selecting child child elements linq It was ['asp.net', 'python']
configuration file silverlight client working wcf polling duplex service It was ['python']
how decompile aspnet website It was ['python']
how to use pycharm debug python script It was ['python']
is way make scrolling viewport box html css It was ['javascript', 'python']
fastestmost efficient app engine local file read memcache hit It was ['python']
md password decryption It was ['python']
watin responding api calls It was ['python']